In [ ]:
#密度map画像作成用コード（disk_cyl-test2の可視化で使ったやつ）

import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt

# -----------------------------
# VTK読み込み
# -----------------------------
fname = "disk.block0.out1.00000.vtk"
grid = pv.read(fname)
rho = grid["dens"]

# -----------------------------
# セル数
# -----------------------------
nx, ny, nz = np.array(grid.dimensions) - 1  # nx=64, ny=256, nz=1
if nz == 0:
    nz = 1  # 2Dの場合

# -----------------------------
# r, phi のセル境界
# -----------------------------
x_edges = np.array(grid.GetXCoordinates())  # 65点
y_edges = np.array(grid.GetYCoordinates())  # 257点

# -----------------------------
# rhoを2D配列に reshape (r, phi)
# -----------------------------
rho_2d = rho.reshape((nx, ny), order='F')  # shape = (64, 256)

# -----------------------------
# pcolormesh用にセル境界に合わせて拡張
# -----------------------------
rho_mesh = np.zeros((nx+1, ny+1))
rho_mesh[:-1, :-1] = rho_2d
rho_mesh[-1, :-1] = rho_2d[-1, :]       # 最後の行をコピー
rho_mesh[:-1, -1] = rho_2d[:, -1]       # 最後の列をコピー
rho_mesh[-1, -1] = rho_2d[-1, -1]       # 右上端もコピー

# -----------------------------
# X-Y座標に変換（セル境界）
# -----------------------------
R, PHI = np.meshgrid(x_edges, y_edges, indexing='ij')
X = R * np.cos(PHI)
Y = R * np.sin(PHI)

# -----------------------------
# プロット
# -----------------------------
plt.figure(figsize=(6,6))
plt.pcolormesh(X, Y, rho_mesh, shading='auto', cmap='viridis')
plt.axis('equal')
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Density (.00000)")
plt.colorbar(label="Density")
plt.savefig("theta_averaged_density_00000.png", dpi=300)  # 高解像度で保存
plt.show()


In [ ]:
#密度グラフ全ステップの画像を作成して保存するコード（disk_cyl-test2の密度mapを作成するときに使ったやつ）

import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import os
import glob
import natsort

# ============================================================
# 設定
# ============================================================
# Athena++ 出力ディレクトリ
outdir = "/home/aian/athena-public-version/results/disk_cyl-test2"
# ファイルパターン
file_pattern = os.path.join(outdir, "disk.block0.out1.*.vtk")
# 保存先ディレクトリ
savedir = os.path.join(outdir, "density_maps")
os.makedirs(savedir, exist_ok=True)

# ============================================================
# ファイル一覧取得（ソート）
# ============================================================
file_list = natsort.natsorted(glob.glob(file_pattern))
if len(file_list) == 0:
    raise FileNotFoundError("VTKファイルが見つかりません。パス設定を確認してください。")
print(f"発見ファイル数: {len(file_list)}")

# ============================================================
# ループ処理
# ============================================================
for i, fname in enumerate(file_list):
    print(f"[{i+1}/{len(file_list)}] {os.path.basename(fname)} を処理中...")

    # -----------------------------
    # VTK読み込み
    # -----------------------------
    grid = pv.read(fname)
    rho = grid["dens"]

    # -----------------------------
    # セル数
    # -----------------------------
    nx, ny, nz = np.array(grid.dimensions) - 1
    if nz == 0:
        nz = 1  # 2Dの場合

    # -----------------------------
    # r, phi のセル境界
    # -----------------------------
    x_edges = np.array(grid.GetXCoordinates())  # r方向
    y_edges = np.array(grid.GetYCoordinates())  # φ方向

    # -----------------------------
    # rhoを2D配列に reshape (r, phi)
    # -----------------------------
    rho_2d = rho.reshape((nx, ny), order='F')

    # -----------------------------
    # pcolormesh用にセル境界に合わせて拡張
    # -----------------------------
    rho_mesh = np.zeros((nx+1, ny+1))
    rho_mesh[:-1, :-1] = rho_2d
    rho_mesh[-1, :-1] = rho_2d[-1, :]
    rho_mesh[:-1, -1] = rho_2d[:, -1]
    rho_mesh[-1, -1] = rho_2d[-1, -1]

    # -----------------------------
    # X-Y座標に変換（セル境界）
    # -----------------------------
    R, PHI = np.meshgrid(x_edges, y_edges, indexing='ij')
    X = R * np.cos(PHI)
    Y = R * np.sin(PHI)

    # -----------------------------
    # プロット
    # -----------------------------
    plt.figure(figsize=(6, 6))
    plt.pcolormesh(X, Y, rho_mesh, shading='auto', cmap='viridis')
    plt.axis('equal')
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(f"Density ({i:05d})")
    plt.colorbar(label="Density")
    plt.tight_layout()

    # 保存
    save_path = os.path.join(savedir, f"density_{i:05d}.png")
    plt.savefig(save_path, dpi=300)
    plt.close()

print(f"\n✅ すべての密度マップを {savedir} に保存しました。")


In [ ]:
#画像を紙芝居形式で動画化するコード（disk_cyl-test2の密度mapを動画化するのに使った）

import matplotlib.animation as animation
import os
import natsort
from PIL import Image

# ============================================================
# 設定
# ============================================================
# 画像ディレクトリ
img_dir = "/home/aian/athena-public-version/results/disk_cyl-test2/density_maps"
# 出力動画ファイル名
output_video = os.path.join(img_dir, "density_movie.mp4")
# フレーム間隔（秒）
interval = 100  # 100ms → 10fps

# ============================================================
# 画像一覧を取得
# ============================================================
img_files = natsort.natsorted([os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".png")])
if len(img_files) == 0:
    raise FileNotFoundError("画像が見つかりません。パスを確認してください。")

# ============================================================
# Figure 作成
# ============================================================
fig, ax = plt.subplots(figsize=(6,6))
ax.axis('off')  # 軸は非表示
im = ax.imshow(Image.open(img_files[0]))

# ============================================================
# 更新関数
# ============================================================
def update(frame):
    im.set_array(Image.open(img_files[frame]))
    return [im]

# ============================================================
# アニメーション作成
# ============================================================
ani = animation.FuncAnimation(fig, update, frames=len(img_files), interval=interval, blit=True)

# ============================================================
# 動画保存
# ============================================================
ani.save(output_video, writer="ffmpeg", dpi=200)
print(f"✅ 動画を保存しました: {output_video}")
）

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import natsort
from PIL import Image

# ============================================================
# 設定
# ============================================================
# 画像ディレクトリ
img_dir = "/home/aian/athena-public-version/results/disk_cyl-test2/density_maps"
# 出力動画ファイル名
output_video = os.path.join(img_dir, "density_movie.mp4")
# フレーム間隔（秒）
interval = 100  # 100ms → 10fps

# ============================================================
# 画像一覧を取得
# ============================================================
img_files = natsort.natsorted([os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".png")])
if len(img_files) == 0:
    raise FileNotFoundError("画像が見つかりません。パスを確認してください。")

# ============================================================
# Figure 作成
# ============================================================
fig, ax = plt.subplots(figsize=(6,6))
ax.axis('off')  # 軸は非表示
im = ax.imshow(Image.open(img_files[0]))

# ============================================================
# 更新関数
# ============================================================
def update(frame):
    im.set_array(Image.open(img_files[frame]))
    return [im]

# ============================================================
# アニメーション作成
# ============================================================
ani = animation.FuncAnimation(fig, update, frames=len(img_files), interval=interval, blit=True)

# ============================================================
# 動画保存
# ============================================================
ani.save(output_video, writer="ffmpeg", dpi=200)
print(f"✅ 動画を保存しました: {output_video}")


In [ ]:
#密度vs半径のグラフを特定のファイルを選んできて同じグラフにプロットするコード（disk_cyl-test3の密度vs半径のグラフを作る特に使った）

import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import os

# ============================================================
# 設定
# ============================================================
outdir = "/home/aian/athena-public-version/results/disk_cyl-test3"
snap_indices = [0, 50, 100]  # 読み込むステップ

plt.figure(figsize=(7, 5))
colors = plt.cm.viridis(np.linspace(0, 1, len(snap_indices)))

# ============================================================
# 各ファイル読み込みループ
# ============================================================
for idx, snap in enumerate(snap_indices):
    fname = os.path.join(outdir, f"disk.block0.out1.{snap:05d}.vtk")

    if not os.path.exists(fname):
        continue

    grid = pv.read(fname)
    rho = grid["dens"]

    # --- 円筒座標では x1 = r ---
    centers = grid.cell_centers().points
    r = centers[:, 0]

    # --- 半径ビン平均 ---
    n_bins = 60
    r_bins = np.linspace(r.min(), r.max(), n_bins + 1)
    r_cent = 0.5 * (r_bins[:-1] + r_bins[1:])
    rho_binned = np.zeros(n_bins)

    for i in range(n_bins):
        mask = (r >= r_bins[i]) & (r < r_bins[i + 1])
        if np.any(mask):
            rho_binned[i] = np.mean(rho[mask])
        else:
            rho_binned[i] = np.nan

    # --- NaNを除外してプロット ---
    valid = ~np.isnan(rho_binned)
    plt.plot(r_cent[valid], rho_binned[valid],
             color=colors[idx],
             label=f"step {snap:05d}",
             alpha=0.8)

# ============================================================
# グラフ設定
# ============================================================
plt.xlabel("Radius r")
plt.ylabel("Density ρ")
plt.xlim(0,2)
plt.ylim(0,6)
plt.title("Radial Density Evolution (00000–00100)")
plt.legend(loc="best", fontsize=8)
plt.grid(True)
plt.tight_layout()
#保存
plt.savefig(os.path.join(outdir, "radial density evolution.png"), dpi=300, bbox_inches="tight")
plt.show()